In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bf
import traceback
import time

def states(main_url):
    main_url = str(main_url)
    r = requests.get(main_url)

    soup = bf(r.content, 'html.parser')
    anchors = soup.find_all("a")

    all_lnks = set()
    for link in anchors:
        if link!= '#':
            link = link.get('href')
            if "policy" not in link and "https" not in link and "#" not in link:

                all_lnks.add("https://www.quicktransportsolutions.com/carrier/"+link)
    for i in list(all_lnks):

        folder_name = f"{i.split('/')[-2]}_{i.split('/')[-1].split('.')[0]}"

        if not os.path.exists(os.path.join(os.getcwd(), folder_name)):
            os.makedirs(folder_name)
    return list(all_lnks)


def get_cities(sub_url_1):
    sub_url_1 = sub_url_1
    r = requests.get(sub_url_1)
    
    soup = bf(r.content, 'html.parser')
    anchors = soup.find_all("a")
    #     print(anchors)
    all_lnks = set()
    for link in anchors:
        link = link.get('href')

        if link != None:

            if "#" not in link and "policy" not in link and "https" not in link and "index" not in link and "carrier" not in link:
                if len((sub_url_1.split('/'))) == 6:



                    all_lnks.add("https://www.quicktransportsolutions.com/carrier/"+sub_url_1.split("/")[-2]+"/"+link)
                else:
                    all_lnks.add("https://www.quicktransportsolutions.com/carrier/"+sub_url_1.split("/")[-3]+"/"+sub_url_1.split("/")[-2]+"/"+link)



    return list(all_lnks)
def cities(url):
    title = []
    address = []
    phone = []
    other_add = []
    USDOT_list = []
    tr = []
    saad = 0
    j = url
    i = 2
    flag = True
#     for new in range(2,70):
    while flag:
        
        if saad == 0:
            r = requests.get(url)
            saad+=1
        else:
            url = f'{j}?page={i}"'
            i+=1
            
            
        
        r = requests.get(url)
        

        soup = bf(r.content, 'html.parser')

        #name
        name_tag = soup.find_all("span", itemprop="name")

        for name in name_tag:
            if name.b != None:
                title.append(name.b.text)

    #     streetAddress

        streetAddress = soup.find_all("span", itemprop="streetAddress")
#         print(len(streetAddress))
        if len(streetAddress) == 0:
            flag = False
            print("This URL has been traversed")
            continue

        for street in streetAddress:
            address.append(street.text)

        phone_tag = soup.find_all('div', {'itemtype': 'https://schema.org/Organization'})

        for k in phone_tag:
            phone_elem = k.find('b', {'itemprop': 'telephone'})
            if phone_elem is not None:
                phone_number = phone_elem.text.strip()
                phone.append(phone_number)
            else:
                phone.append("None")






        regtag = soup.find_all("span", itemprop="addressRegion")
        addressLocality = soup.find_all("span", itemprop="addressLocality")
        postalCode = soup.find_all("span", itemprop="postalCode")
        trucks = soup(text=lambda t: "Trucks" in t.text)



        for reg,add, pos in zip(regtag, addressLocality, postalCode):
            other_add.append(f"{reg.text}, {add.text} {pos.text}")

    #     USDOT

        for USDOT in soup(text=lambda t: "USDOT" in t.text):

            USDOT_list.append(f"{USDOT.text} {USDOT.next_element.text}")

        #Trucs
        trucks = soup(text=lambda t: "Trucks" in t.text)


        for t in trucks:
            tr.append(t)

#         print(len(title), len(address), len(phone), len(other_add), len(tr), len(USDOT_list))

    return title, address, phone, other_add, tr, USDOT_list


def write_file(title, address, x, other_add, y, USDOT_list, file_name, j):
    
    
#     if len(title) == len(address) == len(x[:-1]) ==len(other_add) ==len(y[:-1]) ==len(USDOT_list):
        
    
    df = pd.DataFrame()
    df["Name"] = title
    df["Address"] = address
    df["Phone"] = x
    df["other_add"] = other_add
#   df["Truck_Driver"] = y[:-1]
    df["USDOT"] = USDOT_list

    df.to_excel(f"{file_name}.xlsx")

        

In [ ]:


for i in states(main_url):
    folder_name = f"{i.split('/')[-2]}_{i.split('/')[-1].split('.')[0]}"

    for j in get_cities(i):
        print(j)

        sub_folder = f"{j.split('/')[-1].split('.')[0]}"
        if not os.path.exists(os.path.join(os.getcwd(),folder_name, sub_folder)):
            os.makedirs(f"{folder_name}/{sub_folder}")
        file_name = f"{os.getcwd()}/{folder_name}/{sub_folder}/{sub_folder}"
        if os.path.exists(file_name):
            print("The file already exist: ", file_name)
            continue


        
        try:
            if cities(j) != None:  
                title, address, x, other_add, tr, USDOT_list = cities(j)
                
                
                write_file(title, address, x, other_add, tr, USDOT_list, file_name,j)
        except:
            print(traceback.print_exc())
